## Indus.ai Person and Truck Detection

This notebook contains a step-by-step guide to using Indus.ai's person and truck models!

In [1]:
import os, sys
import time
import boto3
import json
import cv2
import random
import numpy as np
import tensorflow as tf
from sagemaker.content_types import CONTENT_TYPE_JSON, CONTENT_TYPE_CSV, CONTENT_TYPE_NPY
from sagemaker.session import Session

Start the sagemaker runtime session. Set the endpoint name according to what you created in sagemaker

In [2]:
ENDPOINT_NAME = "person-truck-endpoint"
runtime= boto3.Session().client(service_name='runtime.sagemaker')

Read your image file and convert it to bytes.

Then invoke the endpoint. Make sure that your content type is some kind of image [image/jpeg, image/png, image/bmp]

In [3]:
file_name = "people.jpg"
name, extension = os.path.splitext(file_name)

bytes_image = open(file_name, "rb")
bytes_image = bytearray(bytes_image.read())

start_time = time.time()
response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                   ContentType='image/jpeg',
                                   Body=bytes_image)
print("--- %s seconds ---" % (time.time() - start_time))

--- 15.7948420048 seconds ---


The results has a set of bounding boxes under the "detections" key in the form of (xmin, ymin, xmax, ymax, label, score). 

In [4]:
response = response['Body'].read()
results = json.loads(response)['outputs']

Our colour generate will randomly retrieve colours from a pre-defined list of nice looking colours! From there, we'll loop through each of our contours and draw them on the images to show the detections!

In [ ]:
COLOR_OPTIONS = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0],
              [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255],
              [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]

def generate_color():
  color_index = random.randint(0, len(COLOR_OPTIONS)-1)
  return tuple(COLOR_OPTIONS[color_index])

detection_results = json.loads(response)['outputs']['detections']

num_detections = len(detection_results)

frame = cv2.imread(file_name)
height, width, channels = frame.shape
full_mask = frame
output_image = frame.copy()

for index, box in enumerate(detection_results):
    color = generate_color()
    (objX1, objY1, objX2, objY2, label, score) = box
    print("Box = {}".format(box))
    cv2.rectangle(output_image, (int(objX1), int(objY1)), (int(objX2), int(objY2)), color, 2)

cv2.imwrite("output.jpg", output_image)
cv2.imshow("Output", output_image)
cv2.waitKey(0)

Box = [293, 9, 371, 258, u'person', 0.9988898634910583]
Box = [9, 28, 174, 261, u'person', 0.9982282519340515]
Box = [156, 7, 247, 266, u'person', 0.997717022895813]
Box = [381, 27, 504, 261, u'person', 0.997687816619873]
Box = [432, 1, 546, 252, u'person', 0.9967929720878601]
